## set running directory to project source

In [1]:
import os
import numpy as np
import time
os.chdir(os.path.join(os.environ["RNB_PLANNING_DIR"], 'src'))
import matplotlib.pyplot as plt

## 4.1 PlanningScene

##### initialize CombinedRobot and GeometryScene

In [2]:
from pkg.controller.combined_robot import *
from pkg.project_config import *
from pkg.geometry.builder.scene_builder import SceneBuilder

s_builder = SceneBuilder(None)   # create scene builder without detector for virtual scene
crob = CombinedRobot(robots_on_scene=[
    RobotConfig(0, RobotType.indy7gripper, ((0,0,0.01), (0,0,0)), None)]
                     , connection_list=[False])
gscene = s_builder.create_gscene(crob)
gtems = s_builder.add_robot_geometries(color=(0,1,0,0.5), display=True, collision=True)
gscene.show_pose(crob.home_pose)

connection command:
indy0: False
Unable to register with master node [http://localhost:11311]: master may not be running yet. Will keep trying.


Unknown tag "hardwareInterface" in /robot[@name='custom_robots']/transmission[@name='indy0_tran0']/actuator[@name='indy0_motor0']
Unknown tag "hardwareInterface" in /robot[@name='custom_robots']/transmission[@name='indy0_tran1']/actuator[@name='indy0_motor1']
Unknown tag "hardwareInterface" in /robot[@name='custom_robots']/transmission[@name='indy0_tran2']/actuator[@name='indy0_motor2']
Unknown tag "hardwareInterface" in /robot[@name='custom_robots']/transmission[@name='indy0_tran3']/actuator[@name='indy0_motor3']
Unknown tag "hardwareInterface" in /robot[@name='custom_robots']/transmission[@name='indy0_tran4']/actuator[@name='indy0_motor4']
Unknown tag "hardwareInterface" in /robot[@name='custom_robots']/transmission[@name='indy0_tran5']/actuator[@name='indy0_motor5']


Please create a subscriber to the marker
publication OK
published: [0, 0, 0, 0, 0, 0]
Please create a subscriber to the marker


##### add geometries

In [3]:
from pkg.geometry.geometry import *

# add environments (fixed=True for non-movable geometries)
wall = gscene.create_safe(GEOTYPE.BOX, "wall", "base_link", (3,3,0.01), (-0.2,0,0), 
                           rpy=(0,np.pi/2,0), color=(0.8,0.8,0.8,0.5), display=True, fixed=True, collision=True)
floor = gscene.create_safe(GEOTYPE.BOX, "floor", "base_link", (0.72,1.52,0.01), (0.25,0,0), 
                           rpy=(0,0,0), color=(0.8,0.8,0.8,0.5), display=True, fixed=True, collision=True)
wp1 = gscene.create_safe(GEOTYPE.BOX, "wp1", "base_link", (0.1,0.1,0.01), (0.6,-0.3,0),rpy=(0,0,0), 
                         color=(0.8,0.2,0.2,1), display=True, fixed=True, collision=True, parent="floor")
wp2 = gscene.create_safe(GEOTYPE.BOX, "wp2", "base_link", (0.1,0.1,0.01), (0.2,-0.3,0), rpy=(0,0,0), 
                         color=(0.8,0.2,0.2,1), display=True, fixed=True, collision=True, parent="floor")
goal = gscene.create_safe(GEOTYPE.BOX, "goal", "base_link", (0.1,0.1,0.01), (0.3,-0.4,0), 
                          rpy=(0,0,0), color=(0.2,0.2,0.8,1), display=True, fixed=True, collision=True)
 
# add movable (fixed=False for movable geometries)
box1 = gscene.create_safe(GEOTYPE.BOX, "box1", "base_link", (0.05,0.05,0.05), (0.3,0.4,0.031), 
                          rpy=(0,0,0), color=(0.8,0.2,0.2,1), display=True, fixed=False, collision=True)

obstacle = gscene.create_safe(GEOTYPE.BOX, "obstacle", "base_link", (0.05,0.05,0.05), (0.5,0.4,0.031), 
                          rpy=(0,0,0), color=(0.8,0.8,0.2,1), display=True, fixed=False, collision=True)

##### create PlanningScene

In [4]:
from pkg.planning.scene import PlanningScene
pscene = PlanningScene(gscene, combined_robot=crob)

##### create_binder
- Binders (or Actors) are binding points where objects can be attached (or binded)
- Examples are 
  - PlacePlane: plane for object placement
  - Gripper2Tool: 2-finger gripper tool for grasp objects
  - SweepTool: action point to pass waypoints for sweep task

In [5]:
from pkg.planning.constraint.constraint_actor import PlacePlane, Gripper2Tool, SweepFramer

In [6]:
# create PlacePlane on geometry "floor" and "goal"
# when point is not set, the entire upper surface of the geometry becomes valid binding area.
# when point is set, the specific point becomes the only valid binding point.
pscene.create_binder(bname="floor", gname="floor", _type=PlacePlane)
pscene.create_binder(bname="goal", gname="goal", _type=PlacePlane)

In [7]:
# add collision boundary for gripper base
# - set link_name="indy0_tcp" to attach the geometry to end-effector link
# - it can be labeled as fixed=True, as it is "fixed" on the indy0_tcp link
gripper =  gscene.create_safe(GEOTYPE.SPHERE, "grip0", link_name="indy0_tcp", 
                                dims=(0.01,0.01,0.01), center=(0,0,0.14), rpy=(-np.pi/2,0,0), 
                                color=(0.8,0.2,0.2,0.5), display=False, fixed=True, collision=False)

# create Gripper2Tool binder
# Gripper2Tool is a 2-finger gripper, which can rotate along z-direction.
# To align the z-direction with the 2 fingers, rotate by 90 degree along roll axis.
# The gripping point is (0,0,0.11) in local coordinate of "gripper" geometry
pscene.create_binder(bname="grip0", gname="grip0", _type=Gripper2Tool, point=(0,0,0), rpy=(0,0,0))

In [8]:
# Add virtual (no-collision) sweep face. the point is 0.2 m away from the "indy0_tcp" link
# To match the z-direction with the target surface, the geometry is rotated 180 degrees in pitch-axis.
sweep_face =  gscene.create_safe(GEOTYPE.BOX, "sweep_face", link_name="indy0_tcp", 
                                dims=(0.05,0.05,0.001), center=(0,0,0.2), rpy=(0,np.pi,0), 
                                color=(0.2,0.2,0.8,0.1), display=False, fixed=True, collision=False)
                                 
# create SweepTool binder
pscene.create_binder(bname="sweep_face", gname="sweep_face", _type=SweepFramer, point=(0,0,0), rpy=(0,0,0))

##### create_subject
* Subject describes the tasks in the planning scene.
* There are 2 categories in subject:
  1. Object: The object has grip points and placement points for pick&place task
  2. Task: The task is can be any non-physical task. Check SweepLineTask for example
* The subjects can be composed of multiple action points. Examples are:
  1. Grasp2Point: grasping point for 2-finger gripper. 
  2. PlacePoint: The point to place object.
  3. SweepPoint: A waypoint for SweepLineTask.
  * The above 3 action points inherit DirectePoint, for which the orientation is free along z-axis. 
  * If "point" parameter is not set, the entire upper surface is becomes valid action area.

In [9]:
from pkg.planning.constraint.constraint_subject import Grasp2Point, PlacePoint, SweepFrame
from pkg.planning.constraint.constraint_subject import CustomObject, SweepLineTask

In [10]:
## create box object with grasping points along positive & negative y-direction and placement point in the bottom face
box_obj = pscene.create_subject(oname="box1", gname="box1", _type=CustomObject, 
                             action_points_dict = {
                                 "handle1": Grasp2Point("handle1", box1, [0,0,0], [-np.pi/2,0,0]),
                                 "handle2": Grasp2Point("handle2", box1, [0,0,0], [np.pi/2,0,0]),
                                 "bottom": PlacePoint("bottom", box1, [0,0,-0.026], [0,0,0])})

In [11]:
## create box object with grasping points along positive & negative y-direction and placement point in the bottom face
obs_obj = pscene.create_subject(oname="obstacle", gname="obstacle", _type=CustomObject, 
                             action_points_dict = {
                                 "handle1": Grasp2Point("handle1", obstacle, [0,0,0], [-np.pi/2,0,0]),
                                 "handle2": Grasp2Point("handle2", obstacle, [0,0,0], [np.pi/2,0,0]),
                                 "bottom": PlacePoint("bottom", obstacle, [0,0,-0.026], [0,0,0])})

In [12]:
## create sweep task with 2 waypoints
sweep = pscene.create_subject(oname="sweep", gname="floor", _type=SweepLineTask, 
                             action_points_dict = {"wp1": SweepFrame("wp1", wp1, [0,0,0.005], [0,0,0]),
                                                   "wp2": SweepFrame("wp2", wp2, [0,0,0.005], [0,0,0])}
                             )

## Pipeline

In [13]:
from pkg.planning.pipeline import PlanningPipeline
ppline = PlanningPipeline(pscene)

In [14]:
from pkg.ui.ui_broker import *

# start UI
ui_broker = UIBroker.instance()
ui_broker.initialize(ppline, s_builder)
ui_broker.start_server()

ui_broker.set_tables()

Dash is running on http://0.0.0.0:8050/

 * Serving Flask app "pkg.ui.dash_launcher" (lazy loading)


##### initialize_state
* initialize_state(robot_pose) updates robot pose and gets corresponding binding status of current scene.
* state.node of ('floor', 0) means the first subject (object) is placed on the floor and the second subject (sweep) has passed 0 waypoints

In [15]:
initial_state = pscene.initialize_state(crob.home_pose)
print(initial_state.node)

 * Environment: production
('floor', 'floor', 0)   Use a production WSGI server instead.

 * Debug mode: off


## import planners

In [16]:

from pkg.planning.motion.moveit.moveit_planner import MoveitPlanner
mplan = MoveitPlanner(pscene)
from pkg.planning.filtering.grasp_filter import GraspChecker
from pkg.planning.filtering.reach_filter import ReachChecker
from pkg.planning.filtering.latticized_filter import LatticedChecker
from pkg.planning.filtering.task_clearance_filter import TaskClearanceChecker

gcheck = GraspChecker(pscene)
rcheck = ReachChecker(pscene)

checkers_all = [rcheck, gcheck]

# tcheck = TaskClearanceChecker(pscene, gcheck)
# checkers_all = [tcheck, rcheck, gcheck]

# lcheck = LatticedChecker(pscene, gcheck)
# checkers_all.append(lcheck)

## Test pybullet

In [17]:
os.chdir(os.path.join(os.environ["RNB_PLANNING_DIR"], 'src/scripts/developing/pddlstream'))

In [18]:
from convert_pscene import *
from plan_rnb import *

#### load pybullet, convert scene

In [19]:
connect_notebook(use_gui=False)
urdf_pybullet_path = copy_meshes(gscene)
robot_body, body_names, movable_bodies = pscene_to_pybullet(pscene, urdf_pybullet_path, tool_name="grip0", name_exclude_list=["indy0"])
print('Objects:', body_names)
saver = WorldSaver()

[Pybullet] Load urdf from /home/rnb/Projects/rnb-planning/src/robots/custom_robots_pybullet.urdf
[WARING] non-object subject not implemented for now
('Objects:', {1L: 'goal', 2L: 'floor', 3L: 'wall', 4L: 'obstacle', 5L: 'box1'})


### Test with IK checkers

In [47]:
gtimer = GlobalTimer.instance()
gtimer.reset()

In [48]:
log_wFeas = []
for _ in range(10):
    problem = pddlstream_from_problem_rnb(pscene, robot_body, body_names=body_names, 
                                          movable=movable_bodies, checkers=checkers_all,
                                          tool_name="grip0", tool_link_name="indy0_tcp", mplan=mplan)
    _, _, _, stream_map, init, goal = problem
    print('Init:', init)
    print('Goal:', goal)
    print('Streams:', str_from_object(set(stream_map)))
    with Profiler():
        with LockRenderer(lock=not True):
            with gtimer.block("solve"):
                solution = solve(problem, algorithm='adaptive', unit_costs=False, success_cost=INF)
            saver.restore()
    print_solution(solution)
    plan, cost, evaluations = solution
    log_wFeas.append(cost)

Robot: 0
Movable: [4L, 5L]
Fixed: [1, 2, 3]
body 4 - surface 1
body 4 - surface 2
body 4 - surface 3
body 5 - surface 1
body 5 - surface 2
body 5 - surface 3
('Init:', [('CanMove',), ('Conf', q34), ('AtConf', q34), ('HandEmpty',), ('Graspable', 4L), ('Pose', 4L, p49), ('AtPose', 4L, p49), ('Stackable', 4L, 1), ('Stackable', 4L, 2), ('Supported', 4L, p49, 2), ('Stackable', 4L, 3), ('Graspable', 5L), ('Pose', 5L, p50), ('AtPose', 5L, p50), ('Stackable', 5L, 1), ('Stackable', 5L, 2), ('Supported', 5L, p50, 2), ('Stackable', 5L, 3)])
('Goal:', ('and', ('AtConf', q34), ('On', 4L, 1)))
('Streams:', '{TrajCollision, inverse-kinematics, plan-free-motion, plan-holding-motion, sample-grasp, sample-pose, test-cfree-approach-pose, test-cfree-pose-pose, test-cfree-traj-pose}')
Warning! All actions have no cost. Recommend setting unit_costs=True
Warning! Undeclared predicates: ['conf', 'graspable']
Setting negate=True for stream [test-cfree-approach-pose]
Setting negate=True for stream [test-cfree-p

Attempt: 2 | Results: 61 | Depth: 0 | Success: True | Time: 0.189
Stream plan (13, 13, 0.001): [sample-grasp:(4)->(#g11), inverse-kinematics:(4, p52, #g11)->(#q41, #t2759), test-cfree-traj-pose:(#t2759, 4, p52)->(), test-cfree-traj-pose:(#t2759, 5, p53)->(), sample-pose:(4, 1)->(#p15), inverse-kinematics:(4, #p15, #g11)->(#q42, #t2760), test-cfree-traj-pose:(#t2760, 5, p53)->(), test-cfree-pose-pose:(4, #p15, 5, p53)->(), test-cfree-approach-pose:(4, p52, #g11, 5, p53)->(), test-cfree-approach-pose:(4, #p15, #g11, 5, p53)->(), plan-free-motion:(q37, #q41)->(#t2801), plan-free-motion:(#q42, q37)->(#t2803), plan-holding-motion:(#q41, #q42, 4, #g11)->(#t2802)]
Action plan (5, 0.000): [move_free(q37, #q41, #t2801), pick(4, p52, #g11, #q41, #t2759), move_holding(#q41, #q42, 4, #g11, #t2802), place(4, #p15, #g11, #q42, #t2760), move_free(#q42, q37, #t2803)]
iter=0, outs=1) sample-grasp:(4)->[(g46)]
iter=inf, outs=1) inverse-kinematics:(4, p52, g46)->[(q38, c58)]
iter=inf, outs=1) test-cfree-

Attempt: 1 | Results: 24 | Depth: 1 | Success: False | Time: 0.116
Attempt: 2 | Results: 61 | Depth: 0 | Success: True | Time: 0.222
Stream plan (13, 13, 0.001): [sample-grasp:(4)->(#g13), inverse-kinematics:(4, p58, #g13)->(#q45, #t2877), test-cfree-traj-pose:(#t2877, 4, p58)->(), test-cfree-traj-pose:(#t2877, 5, p59)->(), sample-pose:(4, 1)->(#p17), inverse-kinematics:(4, #p17, #g13)->(#q46, #t2878), test-cfree-traj-pose:(#t2878, 5, p59)->(), test-cfree-pose-pose:(4, #p17, 5, p59)->(), test-cfree-approach-pose:(4, p58, #g13, 5, p59)->(), test-cfree-approach-pose:(4, #p17, #g13, 5, p59)->(), plan-free-motion:(#q46, q43)->(#t2919), plan-free-motion:(q43, #q45)->(#t2921), plan-holding-motion:(#q45, #q46, 4, #g13)->(#t2920)]
Action plan (5, 0.000): [move_free(q43, #q45, #t2921), pick(4, p58, #g13, #q45, #t2877), move_holding(#q45, #q46, 4, #g13, #t2920), place(4, #p17, #g13, #q46, #t2878), move_free(#q46, q43, #t2919)]
iter=0, outs=1) sample-grasp:(4)->[(g50)]
iter=inf, outs=0) inverse-k

Streams: [sample-pose:('?o', '?r')->('?p',), sample-grasp:('?o',)->('?g',), inverse-kinematics:('?o', '?p', '?g')->('?q', '?t'), plan-free-motion:('?q1', '?q2')->('?t',), plan-holding-motion:('?q1', '?q2', '?o', '?g')->('?t',)]
Functions: []
Negated: [test-cfree-pose-pose:('?o1', '?p1', '?o2', '?p2')->(), test-cfree-approach-pose:('?o1', '?p1', '?g1', '?o2', '?p2')->(), test-cfree-traj-pose:('?t', '?o2', '?p2')->()]
Optimizers: []

Iteration: 1 | Complexity: 0 | Skeletons: 0 | Skeleton Queue: 0 | Disabled: 0 | Evaluations: 18 | Eager Calls: 0 | Cost: inf | Search Time: 0.000 | Sample Time: 0.000 | Total Time: 0.000
Attempt: 1 | Results: 1 | Depth: 0 | Success: False | Time: 0.007
Stream plan (inf, 0, inf): None
Action plan (inf, inf): None
No plan: increasing complexity from 0 to 1

Iteration: 2 | Complexity: 1 | Skeletons: 0 | Skeleton Queue: 0 | Disabled: 0 | Evaluations: 18 | Eager Calls: 0 | Cost: inf | Search Time: 0.007 | Sample Time: 0.000 | Total Time: 0.007
Attempt: 1 | Result

iter=inf, outs=0) inverse-kinematics:(4, p67, g56)->[]
Sampling while complexity <= 2

Iteration: 4 | Complexity: 2 | Skeletons: 1 | Skeleton Queue: 2 | Disabled: 0 | Evaluations: 27 | Eager Calls: 0 | Cost: inf | Search Time: 0.221 | Sample Time: 0.309 | Total Time: 0.529
Attempt: 1 | Results: 21 | Depth: 0 | Success: False | Time: 0.016
Stream plan (inf, 0, inf): None
Action plan (inf, inf): None
No plan: increasing complexity from 2 to 3
Sampling for up to -0.072 seconds
Sampling while complexity <= 3
iter=1, outs=1) sample-grasp:(4)->[(g57)]
iter=inf, outs=0) inverse-kinematics:(4, p69, g57)->[]
iter=inf, outs=0) inverse-kinematics:(4, p70, g57)->[]

Iteration: 5 | Complexity: 3 | Skeletons: 1 | Skeleton Queue: 3 | Disabled: 0 | Evaluations: 28 | Eager Calls: 0 | Cost: inf | Search Time: 0.239 | Sample Time: 0.340 | Total Time: 0.579
Attempt: 1 | Results: 47 | Depth: 1 | Success: False | Time: 0.108
Attempt: 2 | Results: 157 | Depth: 1 | Success: False | Time: 0.284
Attempt: 3 | Re

Attempt: 2 | Results: 61 | Depth: 0 | Success: True | Time: 0.182
Stream plan (13, 13, 0.001): [sample-grasp:(4)->(#g17), inverse-kinematics:(4, p74, #g17)->(#q58, #t3378), test-cfree-traj-pose:(#t3378, 5, p75)->(), test-cfree-traj-pose:(#t3378, 4, p74)->(), sample-pose:(4, 1)->(#p23), inverse-kinematics:(4, #p23, #g17)->(#q59, #t3379), test-cfree-traj-pose:(#t3379, 5, p75)->(), test-cfree-pose-pose:(4, #p23, 5, p75)->(), test-cfree-approach-pose:(4, p74, #g17, 5, p75)->(), test-cfree-approach-pose:(4, #p23, #g17, 5, p75)->(), plan-free-motion:(q57, #q58)->(#t3421), plan-free-motion:(#q59, q57)->(#t3422), plan-holding-motion:(#q58, #q59, 4, #g17)->(#t3420)]
Action plan (5, 0.000): [move_free(q57, #q58, #t3421), pick(4, p74, #g17, #q58, #t3378), move_holding(#q58, #q59, 4, #g17, #t3420), place(4, #p23, #g17, #q59, #t3379), move_free(#q59, q57, #t3422)]
iter=0, outs=1) sample-grasp:(4)->[(g59)]
iter=inf, outs=0) inverse-kinematics:(4, p74, g59)->[]
Sampling for up to 0.196 seconds
iter=1

Attempt: 1 | Results: 24 | Depth: 1 | Success: False | Time: 0.096
Attempt: 2 | Results: 61 | Depth: 0 | Success: True | Time: 0.222
Stream plan (13, 13, 0.001): [sample-grasp:(4)->(#g18), inverse-kinematics:(4, p77, #g18)->(#q60, #t3465), test-cfree-traj-pose:(#t3465, 5, p78)->(), sample-pose:(4, 1)->(#p24), inverse-kinematics:(4, #p24, #g18)->(#q61, #t3466), test-cfree-traj-pose:(#t3466, 5, p78)->(), test-cfree-traj-pose:(#t3465, 4, p77)->(), test-cfree-pose-pose:(4, #p24, 5, p78)->(), test-cfree-approach-pose:(4, p77, #g18, 5, p78)->(), test-cfree-approach-pose:(4, #p24, #g18, 5, p78)->(), plan-free-motion:(#q61, q60)->(#t3508), plan-free-motion:(q60, #q60)->(#t3509), plan-holding-motion:(#q60, #q61, 4, #g18)->(#t3507)]
Action plan (5, 0.000): [move_free(q60, #q60, #t3509), pick(4, p77, #g18, #q60, #t3465), move_holding(#q60, #q61, 4, #g18, #t3507), place(4, #p24, #g18, #q61, #t3466), move_free(#q61, q60, #t3508)]
iter=0, outs=1) sample-grasp:(4)->[(g77)]
iter=inf, outs=0) inverse-k

Attempt: 1 | Results: 24 | Depth: 1 | Success: False | Time: 0.101
Attempt: 2 | Results: 61 | Depth: 0 | Success: True | Time: 0.199
Stream plan (13, 13, 0.001): [sample-grasp:(4)->(#g19), sample-pose:(4, 1)->(#p25), inverse-kinematics:(4, #p25, #g19)->(#q63, #t3557), test-cfree-traj-pose:(#t3557, 5, p86)->(), inverse-kinematics:(4, p85, #g19)->(#q62, #t3556), test-cfree-traj-pose:(#t3556, 5, p86)->(), test-cfree-traj-pose:(#t3556, 4, p85)->(), test-cfree-pose-pose:(4, #p25, 5, p86)->(), test-cfree-approach-pose:(4, p85, #g19, 5, p86)->(), test-cfree-approach-pose:(4, #p25, #g19, 5, p86)->(), plan-free-motion:(#q63, q63)->(#t3598), plan-free-motion:(q63, #q62)->(#t3599), plan-holding-motion:(#q62, #q63, 4, #g19)->(#t3600)]
Action plan (5, 0.000): [move_free(q63, #q62, #t3599), pick(4, p85, #g19, #q62, #t3556), move_holding(#q62, #q63, 4, #g19, #t3600), place(4, #p25, #g19, #q63, #t3557), move_free(#q63, q63, #t3598)]
iter=0, outs=1) sample-grasp:(4)->[(g85)]
iter=0, outs=1) sample-pose

iter=inf, outs=1) plan-holding-motion:(q70, q72, 4, g90)->[(c106)]
Summary: {complexity: 6, cost: 0.000, evaluations: 65, iterations: 8, length: 2, run_time: 5.576, sample_time: 1.409, search_time: 4.167, skeletons: 2, solutions: 1, solved: True, timeout: False}

Total External Statistics
External: sample-pose | n: 1629 | p_success: 0.956 | overhead: 0.001
External: sample-grasp | n: 2802 | p_success: 0.995 | overhead: 0.002
External: inverse-kinematics | n: 8738 | p_success: 0.205 | overhead: 0.052
External: plan-free-motion | n: 2197 | p_success: 0.744 | overhead: 0.248
External: plan-holding-motion | n: 421 | p_success: 0.953 | overhead: 0.091
External: test-cfree-pose-pose | n: 1236 | p_success: 0.999 | overhead: 0.001
External: test-cfree-approach-pose | n: 2453 | p_success: 1.000 | overhead: 0.001
External: test-cfree-traj-pose | n: 4748 | p_success: 0.852 | overhead: 0.001
Wrote: statistics/py2/kuka-tamp.pkl
         8105130 function calls (7933565 primitive calls) in 5.662 seco

In [51]:
print(gtimer)

solve: 	13582.1 ms/10 = 1358.2 ms (539.776/5724.67)
sample_grasps_4: 	62.7 ms/10 = 6.3 ms (4.733/14.808)
get_stable_4_1: 	15.2 ms/22 = 0.7 ms (0.376/1.418)
ik_fn: 	3149.7 ms/110 = 28.6 ms (0.505/272.904)
check_feas: 	223.8 ms/110 = 2.0 ms (0.497/7.734)
ReachChecker: 	63.4 ms/110 = 0.6 ms (0.198/1.888)
GraspChecker: 	93.9 ms/64 = 1.5 ms (0.546/4.151)
free_motion_gen: 	1206.4 ms/20 = 60.3 ms (36.405/110.387)
holding_motion_gen: 	688.5 ms/10 = 68.9 ms (37.88/182.597)
get_stable_4_2: 	2.5 ms/3 = 0.8 ms (0.455/1.396)



In [52]:
print(log_wFeas)

[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]


### Test without IK checkers

In [58]:
gtimer = GlobalTimer.instance()
gtimer.reset()

In [59]:
log_woFeas = []
for _ in range(10):
    problem = pddlstream_from_problem_rnb(pscene, robot_body, body_names=body_names, 
                                          movable=movable_bodies, checkers=[],
                                          tool_name="grip0", tool_link_name="indy0_tcp", mplan=mplan)
    _, _, _, stream_map, init, goal = problem
    print('Init:', init)
    print('Goal:', goal)
    print('Streams:', str_from_object(set(stream_map)))
    with Profiler():
        with LockRenderer(lock=not True):
            with gtimer.block("solve"):
                solution = solve(problem, algorithm='adaptive', unit_costs=False, success_cost=INF)
            saver.restore()
    print_solution(solution)
    plan, cost, evaluations = solution
    log_woFeas.append(cost)

Robot: 0
Movable: [4L, 5L]
Fixed: [1, 2, 3]
body 4 - surface 1
body 4 - surface 2
body 4 - surface 3
body 5 - surface 1
body 5 - surface 2
body 5 - surface 3
('Init:', [('CanMove',), ('Conf', q74), ('AtConf', q74), ('HandEmpty',), ('Graspable', 4L), ('Pose', 4L, p94), ('AtPose', 4L, p94), ('Stackable', 4L, 1), ('Stackable', 4L, 2), ('Supported', 4L, p94, 2), ('Stackable', 4L, 3), ('Graspable', 5L), ('Pose', 5L, p95), ('AtPose', 5L, p95), ('Stackable', 5L, 1), ('Stackable', 5L, 2), ('Supported', 5L, p95, 2), ('Stackable', 5L, 3)])
('Goal:', ('and', ('AtConf', q74), ('On', 4L, 1)))
('Streams:', '{TrajCollision, inverse-kinematics, plan-free-motion, plan-holding-motion, sample-grasp, sample-pose, test-cfree-approach-pose, test-cfree-pose-pose, test-cfree-traj-pose}')
Warning! All actions have no cost. Recommend setting unit_costs=True
Warning! Undeclared predicates: ['conf', 'graspable']
Setting negate=True for stream [test-cfree-approach-pose]
Setting negate=True for stream [test-cfree-p

Attempt: 1 | Results: 24 | Depth: 1 | Success: False | Time: 0.253
Attempt: 2 | Results: 61 | Depth: 0 | Success: True | Time: 0.365
Stream plan (13, 13, 0.001): [sample-grasp:(4)->(#g21), inverse-kinematics:(4, p98, #g21)->(#q77, #t5273), test-cfree-traj-pose:(#t5273, 4, p98)->(), sample-pose:(4, 1)->(#p29), inverse-kinematics:(4, #p29, #g21)->(#q78, #t5274), test-cfree-traj-pose:(#t5274, 5, p99)->(), test-cfree-traj-pose:(#t5273, 5, p99)->(), test-cfree-pose-pose:(4, #p29, 5, p99)->(), test-cfree-approach-pose:(4, p98, #g21, 5, p99)->(), test-cfree-approach-pose:(4, #p29, #g21, 5, p99)->(), plan-free-motion:(#q78, q77)->(#t5316), plan-free-motion:(q77, #q77)->(#t5317), plan-holding-motion:(#q77, #q78, 4, #g21)->(#t5315)]
Action plan (5, 0.000): [move_free(q77, #q77, #t5317), pick(4, p98, #g21, #q77, #t5273), move_holding(#q77, #q78, 4, #g21, #t5315), place(4, #p29, #g21, #q78, #t5274), move_free(#q78, q77, #t5316)]
iter=0, outs=1) sample-grasp:(4)->[(g93)]
iter=inf, outs=0) inverse-k

iter=inf, outs=0) inverse-kinematics:(4, p104, g96)->[]
iter=1, outs=1) sample-grasp:(4)->[(g97)]
iter=inf, outs=0) inverse-kinematics:(4, p103, g97)->[]
iter=inf, outs=0) inverse-kinematics:(4, p104, g97)->[]

Iteration: 5 | Complexity: 3 | Skeletons: 1 | Skeleton Queue: 3 | Disabled: 0 | Evaluations: 24 | Eager Calls: 0 | Cost: inf | Search Time: 0.262 | Sample Time: 0.906 | Total Time: 1.168
Attempt: 1 | Results: 28 | Depth: 0 | Success: False | Time: 0.019
Stream plan (inf, 0, inf): None
Action plan (inf, inf): None
No plan: increasing complexity from 3 to 4
Sampling for up to -0.625 seconds
Sampling while complexity <= 4
iter=2, outs=1) sample-pose:(4, 1)->[(p105)]
iter=inf, outs=0) inverse-kinematics:(4, p105, g96)->[]
iter=inf, outs=0) inverse-kinematics:(4, p105, g97)->[]
iter=2, outs=1) sample-grasp:(4)->[(g98)]
iter=inf, outs=0) inverse-kinematics:(4, p103, g98)->[]
iter=inf, outs=0) inverse-kinematics:(4, p105, g98)->[]
iter=inf, outs=0) inverse-kinematics:(4, p104, g98)->[]

iter=inf, outs=1) plan-free-motion:(q83, q84)->[(c124)]
iter=inf, outs=1) plan-free-motion:(q85, q83)->[(c125)]
iter=inf, outs=1) plan-holding-motion:(q84, q85, 4, g100)->[(c126)]
Summary: {complexity: 2, cost: 0.000, evaluations: 38, iterations: 3, length: 2, run_time: 0.488, sample_time: 0.297, search_time: 0.190, skeletons: 1, solutions: 1, solved: True, timeout: False}

Total External Statistics
External: sample-pose | n: 1638 | p_success: 0.957 | overhead: 0.001
External: sample-grasp | n: 2812 | p_success: 0.995 | overhead: 0.002
External: inverse-kinematics | n: 8764 | p_success: 0.205 | overhead: 0.052
External: plan-free-motion | n: 2205 | p_success: 0.745 | overhead: 0.247
External: plan-holding-motion | n: 425 | p_success: 0.953 | overhead: 0.091
External: test-cfree-pose-pose | n: 1240 | p_success: 0.999 | overhead: 0.001
External: test-cfree-approach-pose | n: 2461 | p_success: 1.000 | overhead: 0.001
External: test-cfree-traj-pose | n: 4760 | p_success: 0.853 | overhead: 

iter=inf, outs=0) inverse-kinematics:(4, p115, g102)->[]
iter=1, outs=1) sample-pose:(4, 2)->[(p116)]
iter=inf, outs=0) inverse-kinematics:(4, p116, g101)->[]
iter=inf, outs=1) inverse-kinematics:(4, p116, g102)->[(q88, c128)]
iter=inf, outs=1) test-cfree-traj-pose:(c128, 5, p112)->[()]
iter=2, outs=1) sample-grasp:(4)->[(g103)]
iter=inf, outs=0) inverse-kinematics:(4, p115, g103)->[]
iter=inf, outs=0) inverse-kinematics:(4, p116, g103)->[]
iter=2, outs=1) sample-pose:(4, 1)->[(p117)]
iter=inf, outs=0) inverse-kinematics:(4, p117, g101)->[]
iter=inf, outs=1) inverse-kinematics:(4, p117, g102)->[(q89, c129)]
iter=inf, outs=1) test-cfree-traj-pose:(c129, 5, p112)->[()]
iter=inf, outs=0) inverse-kinematics:(4, p113, g103)->[]
iter=inf, outs=0) inverse-kinematics:(4, p114, g103)->[]
iter=inf, outs=0) inverse-kinematics:(4, p117, g103)->[]

Iteration: 6 | Complexity: 3 | Skeletons: 2 | Skeleton Queue: 15 | Disabled: 0 | Evaluations: 43 | Eager Calls: 0 | Cost: inf | Search Time: 0.749 | Sam

Attempt: 2 | Results: 61 | Depth: 0 | Success: True | Time: 0.186
Stream plan (13, 13, 0.001): [sample-grasp:(4)->(#g25), inverse-kinematics:(4, p120, #g25)->(#q90, #t6106), test-cfree-traj-pose:(#t6106, 5, p121)->(), sample-pose:(4, 1)->(#p35), inverse-kinematics:(4, #p35, #g25)->(#q91, #t6107), test-cfree-traj-pose:(#t6107, 5, p121)->(), test-cfree-traj-pose:(#t6106, 4, p120)->(), test-cfree-pose-pose:(4, #p35, 5, p121)->(), test-cfree-approach-pose:(4, p120, #g25, 5, p121)->(), test-cfree-approach-pose:(4, #p35, #g25, 5, p121)->(), plan-free-motion:(q98, #q90)->(#t6148), plan-free-motion:(#q91, q98)->(#t6150), plan-holding-motion:(#q90, #q91, 4, #g25)->(#t6149)]
Action plan (5, 0.000): [move_free(q98, #q90, #t6148), pick(4, p120, #g25, #q90, #t6106), move_holding(#q90, #q91, 4, #g25, #t6149), place(4, #p35, #g25, #q91, #t6107), move_free(#q91, q98, #t6150)]
iter=0, outs=1) sample-grasp:(4)->[(g105)]
iter=inf, outs=0) inverse-kinematics:(4, p120, g105)->[]
Sampling for up to 0.150 se

iter=inf, outs=1) plan-holding-motion:(q102, q103, 4, g109)->[(c150)]
Summary: {complexity: 4, cost: 0.000, evaluations: 38, iterations: 5, length: 2, run_time: 1.077, sample_time: 0.809, search_time: 0.268, skeletons: 1, solutions: 1, solved: True, timeout: False}

Total External Statistics
External: sample-pose | n: 1649 | p_success: 0.956 | overhead: 0.001
External: sample-grasp | n: 2821 | p_success: 0.995 | overhead: 0.002
External: inverse-kinematics | n: 8804 | p_success: 0.206 | overhead: 0.053
External: plan-free-motion | n: 2214 | p_success: 0.746 | overhead: 0.246
External: plan-holding-motion | n: 428 | p_success: 0.953 | overhead: 0.091
External: test-cfree-pose-pose | n: 1245 | p_success: 0.999 | overhead: 0.001
External: test-cfree-approach-pose | n: 2470 | p_success: 1.000 | overhead: 0.001
External: test-cfree-traj-pose | n: 4778 | p_success: 0.853 | overhead: 0.001
Wrote: statistics/py2/kuka-tamp.pkl
         1094929 function calls (1064033 primitive calls) in 1.190 s

Attempt: 2 | Results: 61 | Depth: 0 | Success: True | Time: 0.180
Stream plan (13, 13, 0.001): [sample-grasp:(4)->(#g28), inverse-kinematics:(4, p132, #g28)->(#q96, #t6291), test-cfree-traj-pose:(#t6291, 5, p133)->(), test-cfree-traj-pose:(#t6291, 4, p132)->(), sample-pose:(4, 1)->(#p38), inverse-kinematics:(4, #p38, #g28)->(#q97, #t6292), test-cfree-traj-pose:(#t6292, 5, p133)->(), test-cfree-pose-pose:(4, #p38, 5, p133)->(), test-cfree-approach-pose:(4, p132, #g28, 5, p133)->(), test-cfree-approach-pose:(4, #p38, #g28, 5, p133)->(), plan-free-motion:(#q97, q107)->(#t6334), plan-free-motion:(q107, #q96)->(#t6335), plan-holding-motion:(#q96, #q97, 4, #g28)->(#t6333)]
Action plan (5, 0.000): [move_free(q107, #q96, #t6335), pick(4, p132, #g28, #q96, #t6291), move_holding(#q96, #q97, 4, #g28, #t6333), place(4, #p38, #g28, #q97, #t6292), move_free(#q97, q107, #t6334)]
iter=0, outs=1) sample-grasp:(4)->[(g112)]
iter=inf, outs=1) inverse-kinematics:(4, p132, g112)->[(q108, c156)]
iter=inf, o

Streams: [sample-pose:('?o', '?r')->('?p',), sample-grasp:('?o',)->('?g',), inverse-kinematics:('?o', '?p', '?g')->('?q', '?t'), plan-free-motion:('?q1', '?q2')->('?t',), plan-holding-motion:('?q1', '?q2', '?o', '?g')->('?t',)]
Functions: []
Negated: [test-cfree-pose-pose:('?o1', '?p1', '?o2', '?p2')->(), test-cfree-approach-pose:('?o1', '?p1', '?g1', '?o2', '?p2')->(), test-cfree-traj-pose:('?t', '?o2', '?p2')->()]
Optimizers: []

Iteration: 1 | Complexity: 0 | Skeletons: 0 | Skeleton Queue: 0 | Disabled: 0 | Evaluations: 18 | Eager Calls: 0 | Cost: inf | Search Time: 0.000 | Sample Time: 0.000 | Total Time: 0.000
Attempt: 1 | Results: 1 | Depth: 0 | Success: False | Time: 0.007
Stream plan (inf, 0, inf): None
Action plan (inf, inf): None
No plan: increasing complexity from 0 to 1

Iteration: 2 | Complexity: 1 | Skeletons: 0 | Skeleton Queue: 0 | Disabled: 0 | Evaluations: 18 | Eager Calls: 0 | Cost: inf | Search Time: 0.008 | Sample Time: 0.000 | Total Time: 0.008
Attempt: 1 | Result

Attempt: 1 | Results: 534 | Depth: 0 | Success: False | Time: 0.344
Stream plan (inf, 0, inf): None
Action plan (inf, inf): None
No plan: increasing complexity from 6 to 7
Sampling for up to -3.450 seconds
Sampling while complexity <= 7
iter=5, outs=1) sample-pose:(4, 1)->[(p146)]
iter=inf, outs=0) inverse-kinematics:(4, p146, g117)->[]
iter=inf, outs=0) inverse-kinematics:(4, p146, g120)->[]
iter=inf, outs=0) inverse-kinematics:(4, p146, g118)->[]
iter=inf, outs=0) inverse-kinematics:(4, p146, g119)->[]
iter=inf, outs=0) inverse-kinematics:(4, p141, g121)->[]
iter=inf, outs=0) inverse-kinematics:(4, p145, g121)->[]
iter=inf, outs=0) inverse-kinematics:(4, p146, g121)->[]
iter=inf, outs=0) inverse-kinematics:(4, p142, g121)->[]
iter=inf, outs=0) inverse-kinematics:(4, p143, g121)->[]
iter=inf, outs=0) inverse-kinematics:(4, p144, g121)->[]
iter=6, outs=1) sample-grasp:(4)->[(g122)]
iter=inf, outs=0) inverse-kinematics:(4, p139, g122)->[]

Iteration: 10 | Complexity: 7 | Skeletons: 2 | 

In [60]:
print(gtimer)

solve: 	27775.3 ms/10 = 2777.5 ms (589.42/10772.786)
sample_grasps_4: 	70.1 ms/10 = 7.0 ms (4.82/14.589)
ik_fn: 	18266.7 ms/139 = 131.4 ms (5.632/316.837)
check_feas: 	90.9 ms/139 = 0.7 ms (0.358/2.43)
get_stable_4_1: 	23.7 ms/30 = 0.8 ms (0.412/1.575)
free_motion_gen: 	1282.0 ms/25 = 51.3 ms (7.106/93.529)
holding_motion_gen: 	571.5 ms/12 = 47.6 ms (35.097/64.214)
get_stable_4_2: 	5.1 ms/5 = 1.0 ms (0.739/1.573)
get_stable_4_3: 	0.0 ms/2 = 0.0 ms (0.002/0.004)



In [61]:
print(log_woFeas)

[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]


## play plan

In [23]:
SIMULATE = False
if (plan is None) or not has_gui():
    disconnect()
else:
    command = postprocess_plan(plan)
    if SIMULATE:
#         wait_for_user('Simulate?')
        command.control()
    else:
#         wait_for_user('Execute?')
        #command.step()
        command.refine(num_steps=10).execute(time_step=0.01)

UnsupportedOperation: fileno